## Extract and combine polygon coordinates
This notebook demonstrates how to... 
- extract polygons from baysor geojson file
- convert polygons to coordinates i.e. each cell as multiple sets of coordinates for one polygon
- merge all polygon coordinates for each cell 
- merge all polygon coordinates for each fov 
- merge all polygon coordinates for each run

This output is used to generate gene cell matrix on todata1

In [3]:
options(warn = -1, verbose=FALSE)
#!/usr/bin/env Rscript 
library(dplyr)
library(httr)
library(readr)
library(patchwork)
library(unixtools)
library(ggrepel)
library(repr)
library(purrr)
library(pryr)
set_config(config(ssl_verifypeer = 0L))
ulimit::memory_limit(50000)
set.tempdir("/datastore/lucy/tmp/")
setwd("/datastore/lucy/CosMx")

soft  hard 
50000   Inf

In [2]:
#Load Korsunsky lab functions
source("./R/utils.R")
source("./R/TissueSegFunctions.R")
start_upR(TRUE)

In [3]:
#Path to directory where baysor segmentation output is stored
datadir<-c("/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose")

In [14]:
seg.libs <-list.files(datadir, pattern = "segmentation.csv", full.names = TRUE, recursive = TRUE)
seg.libs

[1] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_297/segmentation.csv"
 [2] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_298/segmentation.csv"
 [3] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_299/segmentation.csv"
 [4] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_300/segmentation.csv"
 [5] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_301/segmentation.csv"
 [6] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_302/segmentation.csv"
 [7] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_49/segmentation.csv" 
 [8] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_50/segmentation.csv" 
 [9] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_51/segmentation.csv" 
[10] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_52/segmentation.csv" 
[11] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_53/segmentation.csv" 
[12] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_54/segmentation.csv" 
[13] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_55/segmentation.csv" 
[14] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_56/segmentation.csv" 
[15] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_57/segmentation.csv" 
[16] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_58/segmentation.csv" 
[17] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_59/segmentation.csv" 
[18] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_60/segmentation.csv" 
[19] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_61/segmentation.csv" 
[20] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_62/segmentation.csv" 
[21] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_63/segmentation.csv" 
[22] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_64/segmentation.csv" 
[23] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_65/segmentation.csv" 
[24] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_66/segmentation.csv" 
[25] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_67/segmentation.csv" 
[26] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_68/segmentation.csv" 
[27] "/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_69/segmentation.csv"

In [5]:
combine_fov_polygons <- function(fov){
    # load polygon file
    sf <- sf::st_read(paste0("/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_", fov,"/segmentation_polygons.json"))
    # extract polygons from geometry collection
    polyg <- st_collection_extract(sf, "POLYGON")
    # name cells by fov 
    polyg$cellID <- rownames(polyg)
    polyg$fov <- paste0("FOV", fov)
    polyg$cellID <- paste0(polyg$fov,"_",polyg$cellID)
    
    # convert multipolygons (shapes) to co-ordinates 
    polygon_coords <- list()
    for(cell in unique(polyg$cellID)){
        #Extract polygon for cell of interest
        multipolygon <- polyg[which(polyg$cellID==cell),c("geometry")]
        #Convert to coord matrix
        cell_polygon_st_coords <- as.data.frame(st_coordinates(multipolygon))
        #Keep only X and Y coordinates from dataframe
        cell_polygon_coords <- cell_polygon_st_coords[,c("X","Y")]
        #Add column with cell barcode
        cell_polygon_coords$cell <- cell
        #Stash in list element
        polygon_coords[[cell]] <- cell_polygon_coords  
    }
    
    # combine the polygons for each cell in fov 
    combined_fov_polygons <- do.call(rbind, polygon_coords)
        write.csv(combined_fov_polygons, paste0("/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_", fov,"/segmentation_polygons.csv"))
}

In [6]:
library(stringr)
for (dir in seg.libs){
    #get fov and number
    fov <- str_extract(dir, "fov_[0-9]+")
    # remove the "fov" part and just keep the number
    fov <- gsub("fov_", "", fov)
    print(paste("Processing FOV:", fov))
    # get stats file path from each
    combine_fov_polygons(fov)
}

[1] "Processing FOV: 297"
Reading layer `segmentation_polygons' from data source 
  `/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_297/segmentation_polygons.json' 
  using driver `GeoJSON'
Simple feature collection with 1 feature and 0 fields
Geometry type: GEOMETRYCOLLECTION
Dimension:     XY
Bounding box:  xmin: 47845.67 ymin: 90966.21 xmax: 52101.13 ymax: 95221.88
Geodetic CRS:  WGS 84
[1] "Processing FOV: 298"
Reading layer `segmentation_polygons' from data source 
  `/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_298/segmentation_polygons.json' 
  using driver `GeoJSON'
Simple feature collection with 1 feature and 0 fields
Geometry type: GEOMETRYCOLLECTION
Dimension:     XY
Bounding box:  xmin: 52101.57 ymin: 90966.43 xmax: 56357.11 ymax: 95181.23
Geodetic CRS:  WGS 84
[1] "Processing FOV: 299"
Reading layer `segmentation_polygons' from data source 
  `/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_299/segmentation_polygons.json' 
  using driver `GeoJSO

In [8]:
fov.list <- list()
for (dir in seg.libs){
    #get fov and number
    fov <- str_extract(dir, "fov_[0-9]+")
    # remove the "fov" part and just keep the number
    fov <- gsub("fov_", "", fov)
    fov.list[[fov]] <- fov
}
fov.list <- unlist(fov.list, use.names=FALSE)

In [9]:
fov.list

[1] "297" "298" "299" "300" "301" "302" "49"  "50"  "51"  "52"  "53"  "54" 
[13] "55"  "56"  "57"  "58"  "59"  "60"  "61"  "62"  "63"  "64"  "65"  "66" 
[25] "67"  "68"  "69"

In [22]:
combine_all_polygons <- function(seg.libs){
# convert multipolygons (shapes) to co-ordinates 
    polygon_coords <- list()
    for(fov in fov.list){
        # Read in polygon .csv
        polygon_coords[[fov]] <- read.csv(paste0("/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/fov_", fov,"/segmentation_polygons.csv"))
    }
    # combine the polygons for each cell in fov 
    all_combined_polygons <- do.call(rbind, polygon_coords)
    all_combined_polygons <- all_combined_polygons %>% dplyr::select(X, Y, cell)
    write.csv(all_combined_polygons, paste0("/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/all_segmentation_polygons.csv"))
    
}

In [23]:
combine_all_polygons(seg.libs)

In [4]:
all_combined_polygons <- read.csv(paste0("/datastore/lucy/6KCosMxTMA/synovium_baysor_cellpose/all_segmentation_polygons.csv"))

In [5]:
head(all_combined_polygons)

,X.1,X,Y,cell
,<dbl>,<dbl>,<dbl>,<chr>
1,297.1,51010.72,91332.84,FOV297_1
2,297.2,51007.98,91336.41,FOV297_1
3,297.3,51024.90,91340.77,FOV297_1
4,297.4,51027.66,91339.87,FOV297_1
5,297.5,51038.25,91346.47,FOV297_1
6,297.6,51044.92,91349.17,FOV297_1
